<a href="https://colab.research.google.com/github/jogamaral/language_detector/blob/master/Rule_based_matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm',disable=['ner'])

In [ ]:
import pandas as pd
df = (pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/Questions.csv',
                  nrows=2_000_000,encoding='ISO-8859-1',usecols=['Title','Id']))
titles = [_ for _ in df.loc[df['Title'].str.lower().str.contains('go')]['Title']]

In [ ]:
df_tags = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/Tags.csv")
go_ids = df_tags.loc[df_tags['Tag'] == 'go']['Id']

def has_go_token(doc):
  for t in doc:
    if t.lower_ in ['go', 'golang']:
      if t.pos_ != 'VERB':
        return True
  return False

all_go_sentences = df.loc[df['Id'].isin(go_ids)]['Title'].tolist()
detectable = [d.text for d in nlp.pipe(all_go_sentences) if has_go_token(d)]

non_detectable = (df.loc[~df['Id'].isin(go_ids)]
                  .loc[df['Title'].str.lower().str.contains('go')]
                  ['Title']
                  .tolist())

non_detectable = [d.text for d in nlp.pipe(non_detectable) if has_go_token(d)]

len(all_go_sentences),len(detectable),len(non_detectable)            

(1858, 942, 112)

In [ ]:
log = []

In [ ]:
model_name = "en_core_web_lg"
model = spacy.load(model_name, disable=["ner"])

def has_go_token(doc):
    for t in doc:
        if t.lower_ in ["go", "golang"]:
            if t.pos_ != "VERB":
                return True
    return False

method = "not-verb-but-pobj"

correct = sum(has_go_token(doc) for doc in model.pipe(detectable))
wrong = sum(has_go_token(doc) for doc in model.pipe(non_detectable))
precision = correct/(correct + wrong)
recall = correct/len(detectable)
accuracy = (correct + len(non_detectable) - wrong)/(len(detectable) + len(non_detectable))

log.append([method,model_name,precision,recall,accuracy]) # this is logged

In [ ]:
pd.DataFrame(data=log,columns=['method','model','precision','recall','accuracy'])

,method,model,precision,recall,accuracy
0,not-verb-but-pobj,en_core_web_md,0.912032,0.957537,0.879507
1,not-verb-but-pobj,en_core_web_sm,0.893738,1.000000,0.893738
